### **Objectives**

1. 실습명 : 
2. 핵심 주제:
    1. Pseudo Labeled 데이터셋 불러오기 및 EDA
    2. base 모델 및 토크나이저 불러오기
    3. Text-to-SQL task 데이터 전처리
    4. Text-to-SQL task 모델 LoRA fine-tuning
3. 학습 목표 :
    1. Pseudo Labeled 데이터셋을 불러오고 EDA를 할 수 있다.
    2. Text-to-SQL task 학습을 위한 base 모델 및 토크나이저를 불러올 수 있다.
    3. Pseudo Labeled 데이터셋의 전처리 전략을 수립할 수 있다.
    4. base 모델에 LoRA fine-tuning을 진행할 수 있다.
4. 학습 개념: 키워드명 :
    1. EDA
    2. Text-to-SQL
    3. LoRA fine-tuning
5. 학습 방향 :
  - Text-to-SQL이라는 특정 task에 대해서 특화시키는 LoRA fine-tuning을 진행합니다. 모델의 성능이 아쉬울 경우 학습 데이터와 적은 GPU 메모리로도 성능을 효율적으로 높일 수 있는 방법에 대해서 체득합니다.
  - 실습 코드는 조교가 직접 구현한 코드를 참고하여 학습합니다.
  - 해당 실습은 모델을 학습시킬 경우 무엇이 필요하고 어떻게 하면 학습을 효율적으로 할 수 있는지 고민해봅니다.


### **Prerequisites**
```
numpy==2.1.0
pandas==2.2.3
transformers==4.56.0
torch==2.8.0+cu126
accelerate==1.10.1
datasets==4.0.0
peft==0.17.1
trl==0.22.2
```

랜덤성을 제어하기 위해 seed를 고정합니다.

In [17]:
import os
import torch
import numpy as np
import random

# tqdm이 텍스트 모드를 사용하도록 설정 (ipywidgets 에러 방지)
os.environ["TQDM_DISABLE"] = "0"
os.environ["TQDM_MININTERVAL"] = "1"

# 시드 설정
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)
torch.cuda.manual_seed(1234)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 1. 데이터셋 불러오기 및 EDA

- 학습 목표 : 데이터셋에 대해서 EDA를 할 수 있다.
- 학습 개념 : EDA
- 진행하는 실습 요약
    - 데이터셋 불러오기
    - 데이터셋 EDA


첨부한 데이터셋을 불러옵니다.

In [1]:
from datasets import load_dataset

text_to_sql = load_dataset('csv', data_files={
    'train': './data/train.csv',
    'test': './data/validation.csv'
})
train_df = text_to_sql["train"].to_pandas()
train_df.head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,['SELECT' 'count' '(' '*' ')' 'FROM' 'head' 'W...,['select' 'count' '(' '*' ')' 'from' 'head' 'w...,['How' 'many' 'heads' 'of' 'the' 'departments'...
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","['SELECT' 'name' ',' 'born_state' ',' 'age' 'F...","['select' 'name' ',' 'born_state' ',' 'age' 'f...","['List' 'the' 'name' ',' 'born' 'state' 'and' ..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","['SELECT' 'creation' ',' 'name' ',' 'budget_in...","['select' 'creation' ',' 'name' ',' 'budget_in...","['List' 'the' 'creation' 'year' ',' 'name' 'an..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,['SELECT' 'max' '(' 'budget_in_billions' ')' '...,['select' 'max' '(' 'budget_in_billions' ')' '...,['What' 'are' 'the' 'maximum' 'and' 'minimum' ...
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,['SELECT' 'avg' '(' 'num_employees' ')' 'FROM'...,['select' 'avg' '(' 'num_employees' ')' 'from'...,['What' 'is' 'the' 'average' 'number' 'of' 'em...


In [3]:
len(set(train_df["db_id"]))

140

column과 각 데이터의 타입을 확인합니다.

### 데이터 필드 설명
- db_id (string): 해당 질문과 관련된 데이터베이스 식별자
- query (string): 질문에 대응하는 SQL 쿼리
- question (string): 원본 영어 자연어 질문
- query_toks (list): SQL 쿼리를 토큰화한 리스트
- query_toks_no_value (list): 값을 제외한 SQL 쿼리 토큰 리스트

`.info()` 매서드를 이용해서 데이터의 타입과 갯수를 파악합니다.

In [19]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7000 entries, 0 to 6999
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   db_id                7000 non-null   object
 1   query                7000 non-null   object
 2   question             7000 non-null   object
 3   query_toks           7000 non-null   object
 4   query_toks_no_value  7000 non-null   object
 5   question_toks        7000 non-null   object
dtypes: object(6)
memory usage: 328.3+ KB


실제 데이터를 찍어보며 어떤 식으로 데이터가 구성되어 있는지 확인합니다.

In [20]:
# 한 줄만 가져오기
for i, row in train_df.iterrows():
    for k, v in row.items():
        print(k, ":", v)
    break

db_id : department_management
query : SELECT count(*) FROM head WHERE age  >  56
question : How many heads of the departments are older than 56 ?
query_toks : ['SELECT' 'count' '(' '*' ')' 'FROM' 'head' 'WHERE' 'age' '>' '56']
query_toks_no_value : ['select' 'count' '(' '*' ')' 'from' 'head' 'where' 'age' '>' 'value']
question_toks : ['How' 'many' 'heads' 'of' 'the' 'departments' 'are' 'older' 'than' '56'
 '?']


저희의 목표는 자연어(text)를 SQL로 변환하는 Text-to-SQL 모델을 학습하는 것입니다.

모델의 입력으로 들어갈 데이터는 자연어이므로 `question`이 되고, 모델의 output으로 나올 데이터는 SQL이 되어야 하므로 `query`가 됩니다.

`db_id`, `query_toks`, `query_toks_no_value` column은 학습 데이터를 분석하는 과정에서는 필요할 수 있지만, 모델을 학습하는 상황에서는 필요한 column이 아니므로 해당 column을 제거하고 column을 재정의합니다.

In [21]:
columns = ["query", "question"]

train_data = train_df[columns]
train_data

,query,question
0,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...
1,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads..."
2,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac..."
3,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...
4,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...
...,...,...
6995,SELECT T1.company_name FROM culture_company AS...,What are all the company names that have a boo...
6996,"SELECT T1.title , T3.book_title FROM movie AS...",Show the movie titles and book titles for all ...
6997,"SELECT T1.title , T3.book_title FROM movie AS...",What are the titles of movies and books corres...
6998,SELECT T2.company_name FROM movie AS T1 JOIN c...,Show all company names with a movie directed i...


모델의 입력으로 들어가는 데이터의 특성을 파악합니다.

NLP에서 중요하게 체크해야 할 EDA 항목은 다음과 같습니다.
1. sequence length
2. language
3. input data domain

위의 사항들을 체크해봅시다.

2번과 3번은 데이터셋의 특성으로 인해 이미 체크가 됩니다.
- language : 영어 및 SQL 코드
- input data domain : 다양한 일반 표(table)에서 뽑은 데이터(Open domain)

In [22]:
# 1. sequence length
max_sequence = [0, 0, 0]
for i, row in train_data.iterrows():
    for i, (k, v) in enumerate(row.items()):
        max_sequence[i] = max(max_sequence[i], len(str(v)))

max_sequence

[577, 224, 0]

`max_length`가 모델의 `max_sequence_length`를 넘지 않을 것으로 예상합니다.

<blockquote>
<b>🧠 모델 입력은 token이니까 token 길이를 측정해야 하는거 아닌가요?</b><br>
맞습니다. 그러나 현재 어떤 모델을 사용할지 정해지지 않은 상황에서는 token 길이를 측정하기는 어렵습니다. 따라서, 대략적인 길이를 보고 모델이 결정되면 token 길이를 다시 측정해야 합니다.
</blockquote>

이제부터 모델과 토크나이저를 불러옵시다.


# 2. base 모델 및 토크나이저 불러오기
- 학습 목표 : base 모델과 토크나이저를 불러올 수 있다.
- 학습 개념 : base 모델
- 진행하는 실습 요약
    - base 모델 불러오기
    - 토크나이저 불러오기

사용할 모델은 [naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B](https://huggingface.co/naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B)입니다.

현재 6GB하의 VRAM에서 학습이 가능한 모델은 위의 모델입니다. 또한, 네이버에서 오픈소스로 공개한 모델이므로, 누구나 사용이 가능한 모델입니다.

모델과 토크나이저를 불러옵니다.

In [15]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "naver-hyperclovax/HyperCLOVAX-SEED-Text-Instruct-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
if tokenizer.pad_token is None: # pad_token 설정이 되어있지 않는 경우가 존재합니다.
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

모델의 max_sequence_length를 확인합니다.

max_position_embeddings의 길이는 position embeddings의 길이를 의미합니다.

position embeddings는 입력 토큰에 순서 정보를 넣는 임베딩이므로 해당 차원이 실제 입력 토큰의 길이와 동일합니다.

In [8]:
max_sequence_length = model.config.max_position_embeddings
max_sequence = [0, 0, 0]
for i, row in train_data.iterrows():
    for i, (k, v) in enumerate(row.items()):
        max_sequence[i] = max(max_sequence[i], len(tokenizer(str(v))["input_ids"]))

print("Max sequence length of model:", max_sequence_length)
print("Max sequence length:", max_sequence)

Max sequence length of model: 8192
Max sequence length: [201, 50, 0]


8192의 길이라면 토크나이징을 한 이후의 `max_length`인 201 + 50 보다 훨씬 크기 때문에 충분히 여유롭습니다.

# 3. Text-to-SQL task 데이터 전처리
- 학습 목표 : chat 형식으로 데이터 전처리를 할 수 있다.
- 학습 개념 : 데이터 전처리
- 진행하는 실습 요약
    - apply_chat_template으로 데이터 전처리하기
    - datasets를 이용하여 Datasets 형식으로 변경하기


허깅페이스 tokenizer에서는 jinja2 포맷의 `chat_template`을 제공합니다.

보통 LLM을 제공하는 회사, 모델마다 `chat_template`이 다릅니다. 또한, 최근에는 `chat_template`을 대부분 지원합니다.

따라서, `chat_template`을 확인하고 `apply_chat_template` 매서드를 이용해서 어떻게 적용이 되는지 확인해봅니다.

In [9]:
if tokenizer.chat_template:
    print("=== chat template 사용 가능 ===\n")
    print(tokenizer.chat_template)
else:
    print("=== chat template 사용 불가능 ===\n")

=== chat template 사용 가능 ===

{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


Jinja2 포맷이 이상하긴 하지만 코드를 어느정도 이해하시는 분들이라면 이게 어떤 의미인지를 대충 유추해볼 수 있습니다.

```jinja
{% for message in messages %}
```
어떠한 `iterable` 객체를 `for loop`를 사용하는 것을 봐서는 `list` 등의 객체로 넣어줘야 할거 같습니다.


```jinja
{{'<|im_start|>' + message['role'] + '' + message['content'] + '<|im_end|>' + ''}}
```

`message` 내에 보면 `role`과 `content`라는 키값이 필요한 것을 알 수 있습니다. 여기서 `dictionary` 등의 객체를 사용하면 될거 같습니다.

또한, 앞뒤에 `<|im_start|>`, `<|im_end|>`와 같은 스페셜 토큰이 붙는 것을 확인하실 수 있습니다. 이 부분은 저희가 건들 필요 없이 알아서 생성해줍니다.

`apply_chat_template`의 입력으로 넣기 위해서는 chat(conversations) 형식을 지켜서 입력으로 넣어줘야 합니다.

기존에 불러온 데이터프레임을 chat 형식으로 만들고 `apply_chat_template`을 적용해보겠습니다.

In [23]:
# 학습용 데이터 전처리

example = train_df[columns].iloc[0]
messages = [
    {
        "role": "user",
        "content": example["question"]
    },
    {
        "role": "assistant",
        "content": example["query"]
    }
]

print(messages)
print()
chat = tokenizer.apply_chat_template(messages, tokenize = False, add_generation_prompt = False)
print(chat)

[{'role': 'user', 'content': 'How many heads of the departments are older than 56 ?'}, {'role': 'assistant', 'content': 'SELECT count(*) FROM head WHERE age  >  56'}]

<|im_start|>user
How many heads of the departments are older than 56 ?<|im_end|>
<|im_start|>assistant
SELECT count(*) FROM head WHERE age  >  56<|im_end|>



보시는 바와 같이 모델의 입력에 맞게 형식을 맞춰서 텍스트를 생성해주는 것을 확인하실 수 있습니다.

주의할 점이 크게 2가지가 있습니다.
1. `assistant`는 정답이기 때문에 학습에서만 입력으로 넣어주고 추론에서는 넣어줘서는 안됩니다.
2. `add_generation_prompt=False`는 `assistant`가 들어갔기 때문에 `False`로 설정합니다. 추론용이라면 `True`로 설정해주셔야 합니다.

In [24]:
## 추론용 데이터 전처리
chat_inference = tokenizer.apply_chat_template(messages[:-1], tokenize = False, add_generation_prompt = True)
print(chat_inference)

<|im_start|>user
How many heads of the departments are older than 56 ?<|im_end|>
<|im_start|>assistant



추론용 데이터를 확인해보시면

`SELECT * FROM players;<|im_end|>`

이 사라진 것을 확인하실 수 있습니다.

유의할 점은
1. 모델이 정답으로 생성할 텍스트 : `SELECT * FROM players;`
2. EOS 스페셜 토큰 : `<|im_end|>`

두가지로 구성되어 있는 `assistant` 부분을 학습에 사용하게 됩니다.


이제 모델 입력으로 넣기 위해서 데이터를 전처리하도록 하겠습니다.

데이터프레임에 있는 데이터를 꺼내서 `Dataset` 객체로 변환하는 전처리를 수행합니다.

In [25]:
import datasets

system_prompt = """You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query."""
user_prompt = """Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<USER_QUERY>
{question}
</USER_QUERY>"""

def convert_to_conversation(examples):
    train_data = []
    for i in range(len(examples)):
        messages = [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt.format(question=examples["question"][i])
            },
            {
                "role": "assistant",
                "content": examples['query'][i]
            }
        ]
        train_data.append(
            {
                "messages": messages,
            }
        )
    return train_data

train_data_list = convert_to_conversation(train_data)
train_dataset = datasets.Dataset.from_list(train_data_list)
print(train_dataset["messages"][0])

[{'content': 'You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query.', 'role': 'system'}, {'content': "Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.\n\n<USER_QUERY>\nHow many heads of the departments are older than 56 ?\n</USER_QUERY>", 'role': 'user'}, {'content': 'SELECT count(*) FROM head WHERE age  >  56', 'role': 'assistant'}]


보통 모델을 학습할 때 사용하는 labels는 inputs와 동일합니다.

(모델은 내부적으로 labels를 한 칸 오른쪽으로 shift한 후 Cross-Entropy loss를 계산합니다. 이는 다음 토큰 예측(next token prediction)과 같은 auto-regressive 분류 작업에서 표준 방식입니다.)

따라서 전처리된 샘플은 다음과 같은 형태가 됩니다:
```python
{
    "input_ids": instruction + model response(assistant),
    "labels": instruction + model response(assistant)
}  # HF 모델이 shift +1 처리를 내부적으로 수행
```

- instruction : 모델에게 입력으로 넣을 지시사항 (실제 User들이 입력으로 넣을 텍스트)
- model response : instruction에 대한 모델의 응답 (`assistant`와 동일)

하지만 우리가 하려는 작업은 instruction 부분을 -100으로 대체하는 것입니다. 왜냐하면 저희는 모델이 정답(`assistant`) 부분만 학습을 하도록 하고 싶기 때문입니다.

```python
{
    "input_ids": instruction + model response(assistant),
    "labels": [-100]*len(instruction) + model response(assistant)
}
```

이렇게 하면 Cross-Entropy 함수에 instruction 토큰은 무시(ignore) 하라고 알려주는 셈입니다. 이를 통해 assistant 부분만 학습을 하도록 만들 수 있습니다.

예를 들어,

```
<|im_start|>system
You are a text to SQL query translator. Users will ask you questions in English and you will generate a SQL query.<|im_end|>
<|im_start|>user
Given the <USER_QUERY>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<USER_QUERY>
How many heads of the departments are older than 56 ?
</USER_QUERY><|im_end|>
<|im_start|>assistant
SELECT count(*) FROM head WHERE age  >  56<|im_end|>
```

위에서 학습해야 할 부분(정답 라벨)은
```
SELECT count(*) FROM head WHERE age  >  56<|im_end|>
```
이 부분이므로, 나머지 부분들은 -100으로 마스킹 처리를 합니다.

In [26]:
def convert_train_data(examples, tokenizer):
    messages = examples["messages"]
    label_message = messages[-1]["content"]
    label_input_ids = tokenizer.encode(
        label_message, add_special_tokens=False, return_tensors="pt"
    ).squeeze(0)

    prompt_input_ids = tokenizer.apply_chat_template(
        messages[:2], add_generation_prompt=False, tokenize=True, return_tensors="pt"
    ).squeeze(0)

    response_start_template_ids = tokenizer.encode("<|im_start|>assistant", return_tensors="pt")[0]

    input_ids = torch.cat(
        [
            prompt_input_ids,
            response_start_template_ids,
            label_input_ids,
            torch.tensor([tokenizer.eos_token_id]),
        ],
        dim=0,
    )
    attention_mask = torch.ones(len(input_ids), dtype=torch.int64)
    labels = torch.cat(
        [
            torch.tensor(
                [-100] * (len(input_ids) - len(label_input_ids) - 1)
            ),  # prompt + label_start_template
            label_input_ids,  # label
            torch.tensor([tokenizer.eos_token_id]),  # [EOS]
        ],
        dim=0,
    )

    return (
        {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }
    )

datasets.disable_progress_bar()
train_dataset = train_dataset.map(
    lambda x: convert_train_data(x, tokenizer),
    batched=False,
    num_proc=1,
    remove_columns=train_dataset.column_names  # 원본 컬럼 제거하고 새 컬럼만 남김
)

train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 7000
})

1. input_ids – 토크나이즈된 시퀀스 데이터
- 토크나이저(tokenizer)가 텍스트를 **서브워드 단위(subword units)**로 분할하고, 이를 **어휘 사전(vocabulary)**에 정의된 **토큰 인덱스(token index)**로 변환합니다.
- 이 시퀀스가 Transformer 모델의 **임베딩 레이어(embedding layer)**에 입력되어, 각 정수가 고차원 벡터로 임베딩됩니다.

2. attention_mask – 시퀀스 마스킹 텐서
- Transformer는 일반적으로 고정된 최대 길이(max sequence length) 입력을 사용합니다.
- 실제 입력 길이가 짧으면 패딩(padding)을 추가해 맞추는데, 이때 패딩 토큰은 self-attention 연산에서 무시되어야 합니다.
- attention_mask는 같은 길이의 바이너리 벡터로,
  - 실제 토큰 위치 → 1
  - 패딩 토큰 위치 → 0
- 이 마스크는 어텐션 스코어(attention score) 계산 시 소프트맥스 이전에 매우 작은 음수(−∞)를 더해, 패딩 위치의 기여도를 완전히 제거합니다.

3. labels – 학습 대상(target) 시퀀스
- labels는 **모델의 출력 로짓(logits)**과 비교할 타겟 시퀀스입니다.
- 언어 모델 학습 시 auto-regressive(next-token prediction) 방식으로 학습하므로, labels는 보통 input_ids와 동일하지만 손실 계산에서 제외할 토큰 위치를 -100으로 마스킹합니다.
- 이렇게 하면 Cross-Entropy Loss 계산 시 무시된 위치는 loss=0으로 처리됩니다.

In [16]:
print("input_ids : ", train_dataset["input_ids"][0])
print("attention_mask : ", train_dataset["attention_mask"][0])
print("labels : ", train_dataset["labels"][0])

input_ids :  [100272, 9125, 198, 2675, 527, 264, 1495, 311, 8029, 3319, 46588, 13, 14969, 690, 2610, 499, 4860, 304, 6498, 323, 499, 690, 7068, 264, 8029, 3319, 13, 100273, 198, 100272, 882, 198, 22818, 279, 366, 6584, 32685, 8226, 7068, 279, 12435, 8029, 3290, 311, 17622, 279, 12974, 828, 11, 13126, 279, 3319, 596, 20047, 11, 53794, 11, 323, 11036, 17413, 382, 27, 6584, 32685, 397, 4438, 1690, 14971, 315, 279, 26280, 527, 9191, 1109, 220, 3487, 18072, 524, 6584, 32685, 29, 100273, 198, 100272, 78191, 4963, 1797, 29771, 4393, 2010, 5401, 4325, 220, 871, 220, 220, 3487, 100275]
attention_mask :  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
labels :  [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

# 4. base model 성능 측정
- 학습 목표 : ko_text2sql 데이터셋을 이용해서 base model의 성능을 측정할 수 있다.
- 학습 개념 : metric, 
- 진행하는 실습 요약
    - 테스트 데이터셋 불러오기
    - 성능 측정

model 학습을 하기 전에 base model의 성능이 얼마인지를 측정합니다.

모델의 성능을 측정하기 위해서는 2가지가 필요합니다.
1. 성능을 측정하기 위한 테스트 데이터셋
2. 성능을 측정하기 위한 지표

ko_text2sql에 있는 test 데이터셋을 활용하겠습니다.

In [17]:
test_df = text_to_sql["test"].to_pandas()
print(len(test_df))
test_df.head()

1034


,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,concert_singer,SELECT count(*) FROM singer,How many singers do we have?,['SELECT' 'count' '(' '*' ')' 'FROM' 'singer'],['select' 'count' '(' '*' ')' 'from' 'singer'],['How' 'many' 'singers' 'do' 'we' 'have' '?']
1,concert_singer,SELECT count(*) FROM singer,What is the total number of singers?,['SELECT' 'count' '(' '*' ')' 'FROM' 'singer'],['select' 'count' '(' '*' ')' 'from' 'singer'],['What' 'is' 'the' 'total' 'number' 'of' 'sing...
2,concert_singer,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere...","['SELECT' 'name' ',' 'country' ',' 'age' 'FROM...","['select' 'name' ',' 'country' ',' 'age' 'from...","['Show' 'name' ',' 'country' ',' 'age' 'for' '..."
3,concert_singer,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev...","['SELECT' 'name' ',' 'country' ',' 'age' 'FROM...","['select' 'name' ',' 'country' ',' 'age' 'from...","['What' 'are' 'the' 'names' ',' 'countries' ',..."
4,concert_singer,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ...","['SELECT' 'avg' '(' 'age' ')' ',' 'min' '(' 'a...","['select' 'avg' '(' 'age' ')' ',' 'min' '(' 'a...","['What' 'is' 'the' 'average' ',' 'minimum' ','..."


학습 데이터셋과 동일한 성격을 가진 데이터셋입니다.

따라서, 테스트 데이터셋도 동일하게 필요한 column만 남기도록 하겠습니다.

In [18]:
test_data = test_df[columns]
test_data

,query,question
0,SELECT count(*) FROM singer,How many singers do we have?
1,SELECT count(*) FROM singer,What is the total number of singers?
2,"SELECT name , country , age FROM singer ORDE...","Show name, country, age for all singers ordere..."
3,"SELECT name , country , age FROM singer ORDE...","What are the names, countries, and ages for ev..."
4,"SELECT avg(age) , min(age) , max(age) FROM s...","What is the average, minimum, and maximum age ..."
...,...,...
1029,SELECT Citizenship FROM singer WHERE Birth_Yea...,What are the citizenships that are shared by s...
1030,SELECT count(*) FROM Other_Available_Features,How many available features are there in total?
1031,SELECT T2.feature_type_name FROM Other_Availab...,What is the feature type name of feature AirCon?
1032,SELECT T2.property_type_description FROM Prope...,Show the property type descriptions of propert...


In [19]:
text_idx = 0

test_input = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt.format(question=test_data["question"][text_idx])},
]
print("정답 :", test_data["query"][text_idx])

정답 : SELECT count(*) FROM singer


이제 base model의 성능을 측정하기 위해 측정 metric을 설정합니다. 저희가 사용할 metric은 `execution accuracy`입니다.

text to sql task의 경우, 자연어를 입력으로 받아 model이 sql문을 출력하는 것이므로, 모델의 response 결과물이 실제 sql문이 잘 동작하는지를 기준으로 평가를 진행할 수 있습니다. 이러한 평가 방식은 sql문은 다르더라도 의도대로 실행이 되었다면 정답이 될 수 있으므로 정확히 SQL문이 일치하는지 평가하는 exact match 보다 유연한 방식입니다.

이러한 metric은 실제로 database에 들어있는 데이터를 sql문을 실행했을 때, 의도한 sql문이 동작하는지를 평가하는 방식입니다. 아래 예시 데이터로 진행해보겠습니다.

In [6]:
!cd test-suite-sql-eval-master && python evaluation.py --gold evaluation_examples/gold.txt --pred evaluation_examples/predict.txt --db database/ --etype exec --plug_value

Evaluating 10th prediction
Evaluating 20th prediction
Evaluating 30th prediction
Evaluating 40th prediction
Evaluating 50th prediction
Evaluating 60th prediction
Evaluating 70th prediction
Evaluating 80th prediction
Evaluating 90th prediction
Evaluating 100th prediction
Evaluating 110th prediction
Evaluating 120th prediction
Evaluating 130th prediction
                     easy                 medium               hard                 extra                all                  joint_all           
count                146                  106                  38                   32                   322                  132                 
=====================   EXECUTION ACCURACY     =====================
execution            0.164                0.038                0.000                0.031                0.090                0.008               


                     turn 1               turn 2               turn 3               turn 4               turn > 4            
count  

예시 결과물을 보시면 all이 execution에서 0.090 인 것을 확인하실 수 있습니다. 모델의 예측 결과물을 100개를 실행했다면 9개만 의도한대로 실행되었다는 것과 동일한 의미입니다.

이번에는 학습하지 않은 base 모델을 가지고 얼마나 성능이 나오고 있는지 측정해보겠습니다. 모델 추론을 돌리는데 시간이 걸리므로 test 데이터 중에서 100개만 미리 모델 추론을 실행했습니다. 따로 실행을 해보고 싶으시다면 `python inference --pred_file_name base_model_predict.txt`로 실행하면 base model로 추론을 돌리실 수 있습니다. (주의! 모델을 학습한 이후에 돌리면 학습한 모델로 동작하므로 유의하시길 바랍니다.)

In [ ]:
!cd test-suite-sql-eval-master && python evaluation.py --gold gold.txt --pred base_model_predict.txt --db database/ --etype exec --plug_value

                     easy                 medium               hard                 extra                all                 
count                12                   50                   21                   17                   100                 
=====================   EXECUTION ACCURACY     =====================
execution            0.417                0.000                0.000                0.000                0.050               


# 5. Text-to-SQL task 모델 LoRA fine-tuning

LoRA 설정과 Trainer 설정을 하고 모델 학습을 진행합니다.

LoRA config 설정을 합니다. 자세한 설정은 [Huggingface LoRA Config](https://huggingface.co/docs/peft/package_reference/lora#peft.LoraConfig)를 참고해주세요.

In [11]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]  # safe default for many LLMs
)

### SFTConfig 주요 설정 설명
각 설정들은 기본적인 AI에 대한 지식이 필요한 부분들이 많습니다. 따라서, 각 설정들은 하나하나 블로그 글을 참고해가며 깊이 이해해주세요.

1. output_dir
- 설명: 학습이 끝난 모델 가중치, 체크포인트, 로그를 저장할 디렉터리 (또는 Hugging Face Hub 저장소 이름)
- 중요성: 실험 결과를 재현하거나 이어서 학습하려면 필수로 설정해야 함

2. max_seq_length
- 설명: 한 학습 샘플의 최대 토큰 길이
- 영향:
	1. 길이를 크게 하면 더 긴 문장을 학습할 수 있으나 GPU 메모리 사용량 증가
	2. 너무 작으면 긴 문장이 잘려서 학습 품질 저하

3. packing
	- 설명: 여러 짧은 샘플을 하나의 시퀀스로 이어 붙여(max_seq_length까지) 학습
	- 장점: GPU 메모리 효율 ↑, 학습 속도 ↑ (패딩 낭비 감소)
	- 주의점: 시퀀스 사이를 구분하는 special token(EOS 등)을 넣어야 데이터 섞임 방지
  - PACKING 예시

    데이터셋 샘플이 각각 독립된 시퀀스로 학습되고 학습할 토큰들이 짧으면 패딩이 많아집니다.

    ```
    샘플 1: [USER] 오늘 날씨 어때? [EOS] [PAD] [PAD] [PAD] ...
    샘플 2: [USER] 내일 비 와? [EOS] [PAD] [PAD] [PAD] ...
    샘플 3: [USER] 주말 일정 보여줘 [EOS] [PAD] [PAD] ...
    ```
    GPU는 [PAD]도 연산해야 하므로 메모리 낭비 & 속도 저하가 발생합니다.

    짧은 샘플들을 이어붙이고, 샘플 사이에 EOS 같은 구분 토큰을 추가하여 최대 길이(max_seq_length)에 맞게 하나의 긴 시퀀스로 묶습니다.

    Packed 시퀀스:
    ```
    [USER] 오늘 날씨 어때? [EOS]
    [USER] 내일 비 와? [EOS]
    [USER] 주말 일정 보여줘 [EOS]
    ```
    - 패딩이 거의 없고 GPU 연산 효율 ↑
    - 한 시퀀스 안에서 여러 예시를 한 번에 학습 → 학습 속도 ↑
    - `Packing=False` : `[샘플1][PAD PAD PAD] [샘플2][PAD PAD PAD]`
    - `Packing=True` : `[샘플1][EOS][샘플2][EOS][샘플3][EOS]	`

4. num_train_epochs
- 설명: 전체 데이터셋을 몇 번 반복해서 학습할지 결정
- 팁: LoRA/QLoRA는 적은 Epoch(2~3)으로도 잘 수렴하는 경우가 많음

5. per_device_train_batch_size
- 설명: GPU 한 장(또는 장치 하나)에서 한 번에 처리할 샘플 개수
- 조절 포인트: GPU 메모리에 맞춰 조절. 작게 하면 gradient_accumulation_steps로 보완 가능

6. gradient_accumulation_steps
- 설명: 여러 스텝의 그래디언트를 누적해 한 번만 역전파/가중치 업데이트
- 장점: 작은 배치로도 큰 effective batch size 구현 가능
- 예시: batch_size=1, grad_accum=4 → 실제 batch size = 4와 비슷한 효과
- `gradient_accumulation_steps`은 [링크]()를 참고해주세요.

7. gradient_checkpointing
- 설명: 순전파(forward) 중 일부 중간 값을 저장하지 않고 필요할 때 다시 계산
- 장점: GPU 메모리 절약 (특히 대형 모델 학습 시 필수)
- 단점: 연산량 증가 → 학습 속도 소폭 느려짐

8. optim
- 설명: 사용할 옵티마이저 선택 (adamw_torch_fused는 PyTorch의 fused AdamW)
- 장점: 일반 AdamW보다 빠르고 메모리 효율 좋음 (지원 GPU 필요)

9. logging_steps
- 설명: 몇 step마다 학습 로그를 기록할지
- 팁: 너무 작으면 로그 과다, 너무 크면 학습 추적 힘듦 → 10~50 step 정도 권장

10. save_strategy
- 설명: 모델 체크포인트 저장 시점 (epoch, steps, no)
- 예시: "epoch" → 각 epoch가 끝날 때마다 저장

11. learning_rate
- 설명: 모델 학습률
- 팁: QLoRA 논문에서 2e-4가 안정적이라고 권장
- 주의: 지나치게 높으면 loss 폭주, 낮으면 학습 느림

12. fp16 / bf16
- 설명: 학습 시 연산 정밀도 선택
- fp16: 대부분 GPU 지원, 메모리 절약, 속도 빠름
- bf16: 최신 GPU(Ampere 이상)에서 권장, 안정성 더 좋음
- 팁: Colab A100/T4 → fp16 / A100(Ampere) 이상 → bf16 추천

13. max_grad_norm
- 설명: Gradient clipping 값 (너무 큰 gradient를 잘라 학습 안정화)
- QLoRA 권장값: 0.3

14. warmup_ratio
- 설명: 학습 초기에 learning rate를 천천히 올리는 비율
- 장점: 학습 안정화, 초기 손실 폭주 방지
- QLoRA 논문 값: 0.03 (~3% 단계는 warmup)

15. lr_scheduler_type
- 설명: 학습률 스케줄링 방식
- constant: 학습 내내 일정한 학습률
- (다른 옵션: linear, cosine 등)

16. push_to_hub
- 설명: 학습이 끝난 모델을 Hugging Face Hub에 자동 업로드할지 여부
- 협업/공유: 팀원이나 공개 프로젝트에 유용

17. report_to
- 설명: 학습 메트릭을 어디로 보낼지 (예: "tensorboard", "wandb")
- 장점: 학습 과정 시각화 가능

학습 설정을 진행합니다.

In [ ]:
from trl import SFTConfig, SFTTrainer

output_dir = "outputs"
num_train_epochs=1
per_device_train_batch_size = 1 # GPU가 부족하다면 해당 값을 줄여주세요.
gradient_accumulation_steps = 4
warmup_ratio = 0.03
max_steps = 1000 # 학습 시간이 오래 걸린다면 해당 값을 줄여주세요.
learning_rate = 2e-5
logging_steps = 1
weight_decay = 0.01
max_grad_norm=1.0
lr_scheduler_type = "linear"
report_to = "none" # Use this for WandB etc
bf16=False
gradient_checkpointing=False
optim="adamw_torch"

train_cfg = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    lr_scheduler_type=lr_scheduler_type,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    logging_steps=logging_steps,
    save_steps=max_steps,
    max_steps=max_steps,
    fp16=False if bf16 else True,
    bf16=bf16,
    report_to=report_to,
    gradient_checkpointing=gradient_checkpointing,
    optim=optim,
)

학습을 진행합니다.

In [30]:
trainer = SFTTrainer(
    model = model,
    processing_class = tokenizer,
    train_dataset = train_dataset.select(range(max_steps * per_device_train_batch_size)), # max_steps * batch_size 만큼만 학습합니다.
    peft_config=peft_config,
    args = train_cfg,
)

c:\Users\SSAFY\lucas\skeleton-2\.venv\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\Users\SSAFY\lucas\skeleton-2\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [31]:
trainer_stats = trainer.train()
print(trainer_stats)

Step,Training Loss
1,4.540800
2,3.745800
3,2.759600
4,2.015100
5,3.349700
6,2.219400
7,2.117100
8,2.479700
9,1.619300
10,1.745400


TrainOutput(global_step=100, training_loss=0.8179761552810669, metrics={'train_runtime': 93.8791, 'train_samples_per_second': 4.261, 'train_steps_per_second': 1.065, 'total_flos': 126295603765248.0, 'train_loss': 0.8179761552810669, 'epoch': 4.0})


LoRA를 저장하는 방식은 두가지가 있습니다.
1. base model에 LoRA weight를 merge해서 저장하는 방식
2. LoRA weight만 따로 저장하는 방식

두 방식 모두 가능하지만, 여기서는 2번 방식으로 진행하겠습니다. 많은 학습을 돌린다면 1번보다는 2번이 더욱 비용 효율적입니다.

In [32]:
# ============================================
# Save LoRA adapter (small)
# ============================================
merge_and_save = False
if merge_and_save:
    trainer.model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    print(f"LoRA adapter saved to: {output_dir}")

# ============================================
# (Optional) Merge LoRA into base weights and save a full model
# WARNING: creates a large model; only do this if you need standalone weights
# ============================================
if merge_and_save:
    from peft import AutoPeftModelForCausalLM
    merged_model = AutoPeftModelForCausalLM.from_pretrained(
        output_dir,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float16,
        device_map="auto",
    )
    merged_model = merged_model.merge_and_unload()
    merged_dir = output_dir + "-merged"
    merged_model.save_pretrained(merged_dir, safe_serialization=True)
    tokenizer.save_pretrained(merged_dir)
    print(f"Merged full model saved to: {merged_dir}")


모델 결과물이 잘 나오는지 테스트를 진행합니다. 우선 학습 데이터로 학습이 잘 되었는지 확인만 해보고 실제 모델의 테스트는 평가를 통해 진행합니다.

In [34]:
text_idx = len(train_df) - 1

test_input = [
    {'role': 'system', 'content': system_prompt},
    {'role': 'user', 'content': user_prompt.format(question=train_df["question"][text_idx])},
]
print("정답 :", train_df["query"][text_idx])

inputs = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, return_dict=True, return_tensors="pt")
input_prompt = tokenizer.apply_chat_template(test_input, add_generation_prompt=True, tokenize=False)
with torch.no_grad():
    output_ids = model.generate(
        **inputs.to("cuda"),
        max_new_tokens=128,
        stop_strings=["<|endofturn|>", "<|stop|>"],
        tokenizer=tokenizer
    )

print(tokenizer.batch_decode(output_ids)[0][len(input_prompt) - 1:])

정답 : SELECT T2.company_name FROM movie AS T1 JOIN culture_company AS T2 ON T1.movie_id  =  T2.movie_id WHERE T1.year  =  1999

SELECT T1.company_name FROM company AS T1 JOIN movie_directors AS T2 ON T1.company_id = T2.company_id WHERE T2.movie_year = 1999<|endofturn|>


이제 모델 성능을 평가해봅시다.

모델 성능을 평가하기 이전에 jupyter notebook은 커널을 계속 실행하고 있기 때문에 jupyter notebook을 restart를 해야만 jupyter notebook에서 사용하는 GPU가 비워집니다. 꼭 jupyter notebook을 비우고 `python inference.py` 명령어를 실행해주세요. `outputs/` 디렉토리가 있다면 LoRA 모델을 먼저 불러오게끔 설정이 되어 있습니다.

inference를 돌린 이후에는 `cd test-suite-sql-eval-master && python evaluation.py --gold gold.txt --pred predict.txt --db database/ --etype exec --plug_value` 명령어로 평가를 직접 진행해보시길 바랍니다.